In [9]:
import pandas as pd
from pandas_datareader import data, wb
import matplotlib.dates as dates
import datetime

In [10]:
start = pd.to_datetime('2012-01-01')
end = pd.to_datetime('2020-10-12')

In [11]:
tesla = data.DataReader('AAPL', 'yahoo', start, end)

In [12]:
tesla

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2012-01-03,14.732142,14.607142,14.621428,14.686786,302220800.0,12.691425
2012-01-04,14.810000,14.617143,14.642858,14.765715,260022000.0,12.759631
2012-01-05,14.948215,14.738214,14.819643,14.929643,271269600.0,12.901293
2012-01-06,15.098214,14.972143,14.991786,15.085714,318292800.0,13.036158
2012-01-09,15.276786,15.048214,15.196428,15.061786,394024400.0,13.015480
...,...,...,...,...,...,...
2020-10-06,116.120003,112.250000,115.699997,113.160004,161498200.0,113.160004
2020-10-07,115.550003,114.129997,114.620003,115.080002,96849000.0,115.080002
2020-10-08,116.400002,114.589996,116.250000,114.970001,83477200.0,114.970001


# Ichimoku Application

In [13]:
# Tenkan-sen (Conversion Line): (9-period high + 9-period low)/2
period9_high = tesla['High'].rolling(window=9).max()
period9_low = tesla['Low'].rolling(window=9).min()
tesla['tenkan_sen'] = (period9_high + period9_low)/2

# Kijun-sen (Base Line): (26-period high + 26-period low)/2
period26_high = tesla['High'].rolling(window=26).max()
period26_low = tesla['Low'].rolling(window=26).min()
tesla['kijun_sen'] = (period26_high + period26_low)/2 

# Senkou Span A (Leading Span A): (Conversion line + Base line)/2
tesla['senkou_span_a'] = ((tesla['tenkan_sen'] + tesla['kijun_sen'])/2).shift(26)

# Senkou Span B (Leading Span B): (52-period high + 52-period low)/2
period52_high = tesla['High'].rolling(window=52).max()
period52_low = tesla['Low'].rolling(window=52).min()
tesla['senkou_span_b'] = ((period52_high + period52_low)/2).shift(26)

# (Chikou Span): The most current closing price plotted 26 time periods behind (Optional)
tesla['chikou_span'] = tesla['Close'].shift(-26)

In [14]:
tesla

,High,Low,Open,Close,Volume,Adj Close,tenkan_sen,kijun_sen,senkou_span_a,senkou_span_b,chikou_span
Date,,,,,,,,,,,
2012-01-03,14.732142,14.607142,14.621428,14.686786,302220800.0,12.691425,NaN,NaN,NaN,NaN,17.613214
2012-01-04,14.810000,14.617143,14.642858,14.765715,260022000.0,12.759631,NaN,NaN,NaN,NaN,17.622143
2012-01-05,14.948215,14.738214,14.819643,14.929643,271269600.0,12.901293,NaN,NaN,NaN,NaN,17.950001
2012-01-06,15.098214,14.972143,14.991786,15.085714,318292800.0,13.036158,NaN,NaN,NaN,NaN,18.195000
2012-01-09,15.276786,15.048214,15.196428,15.061786,394024400.0,13.015480,NaN,NaN,NaN,NaN,17.773928
...,...,...,...,...,...,...,...,...,...,...,...
2020-10-06,116.120003,112.250000,115.699997,113.160004,161498200.0,113.160004,111.360001,120.539997,115.180626,107.536251,NaN
2020-10-07,115.550003,114.129997,114.620003,115.080002,96849000.0,115.080002,112.695000,120.539997,117.714375,108.643749,NaN
2020-10-08,116.400002,114.589996,116.250000,114.970001,83477200.0,114.970001,114.970001,120.539997,119.645000,110.543751,NaN


## Algorithm vizualization

In [15]:
import plotly.graph_objs as go

In [16]:
#declare figure
fig = go.Figure()

#Set up traces
fig.add_trace(go.Scatter(x=tesla.index, y= tesla['tenkan_sen'],line=dict(color='royalblue', width=.8), name = 'Tenkan Sen'))
fig.add_trace(go.Scatter(x=tesla.index, y= tesla['kijun_sen'],line=dict(color='orange', width=.8), name = 'Kijun Sen'))
fig.add_trace(go.Scatter(x=tesla.index, y= tesla['senkou_span_a'],line=dict(color='black', width=.8), name = 'Senkou_span_a'))
fig.add_trace(go.Scatter(x=tesla.index, y= tesla['senkou_span_b'],line=dict(color='purple', width=1.8), name = 'Senkou_span_b',fill='tonexty',fillcolor='GREEN',opacity=.1))
fig.add_trace(go.Scatter(x=tesla.index, y= tesla['chikou_span'],line=dict(color='red', width=.8), name = 'Chinkou_span'))

fig.add_trace(go.Candlestick(x=tesla.index,
                open=tesla['Open'],
                high=tesla['High'],
                low=tesla['Low'],
                close=tesla['Close'], name = 'market data'))

# Add titles
fig.update_layout(
    title='Ichimoku Strategy',
    yaxis_title='Tesla Stock (USD per Shares)')

# X-Axes
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)

#Show
fig.show()